In [1]:
import simpy
import numpy as np
import scipy.stats as st
import pandas as pd
import copy

from simulation import Person,simulate

## Preguntas
- Cual es la media de la cantidad de tareas resultas siguiendo cada estrategia?

- Comparar que estrategia de administración de tiempo es mas productiva?
  - 24-6
  - 12-3
  - Free

- ¿Cuál es la distribución de la cantidad de tareas completadas en la ventana de tiempo de 480 minutos? Esto te permitiría entender cuánto varía el rendimiento de la persona.

- ¿Cómo afecta la duración de los descansos a la cantidad de tareas completadas? Podrías comparar las simulaciones con diferentes duraciones de descanso para ver si hay alguna diferencia significativa.

- ¿Cómo afectan las interrupciones a la cantidad de tareas completadas? Similar a la pregunta anterior, podrías comparar las simulaciones con y sin interrupciones.

- ¿Cuál es la distribución del tiempo entre tareas completadas? Esto te permitiría entender cómo se distribuye el tiempo de trabajo de la persona.

- ¿Cómo se correlacionan el número de descansos y el número de tareas completadas? Esto podría ayudarte a entender si tomar más descansos lleva a completar más tareas.

In [61]:
import copy

def get_prob(n, time, b_time=0, b_duration=0, interruptions=True):
    prob = pd.DataFrame({})
    
    for i in range(n):
        env = simpy.Environment()
        person = Person(env,'S', b_time, b_duration,interr= interruptions)
        env.run(until=time)
        env.event().succeed()
        env.event().succeed()
        env.event().succeed()
        
        # Data Collecting
        prob_row = pd.DataFrame({
            "work_to_work": [copy.copy(person.work_to_work)],
            "work_to_break": [copy.copy(person.work_to_break)],
            "work_to_interrupt": [copy.copy(person.work_to_interrupt)],
            
            "break_to_work": [copy.copy(person.break_to_work)],
            "break_to_interrupt": [copy.copy(person.break_to_interrupt)],
            
            "interrupt_to_work": [copy.copy(person.interrupt_to_work)],
            "interrupt_to_break": [copy.copy(person.interrupt_to_break)]
        })
        
        
        prob = pd.concat([prob,prob_row])

    aux = prob.apply(sum, axis=0)
    print(aux.keys())
    work_to_break = aux["work_to_break"]
    work_to_interrupt = aux["work_to_interrupt"]
    
    twork =  work_to_break + work_to_interrupt
    
    break_to_work = aux["break_to_work"]
    break_to_interrupt = aux["break_to_interrupt"]
    
    tbreak = break_to_work + break_to_interrupt
    
    interrupt_to_work = aux["interrupt_to_work"]
    interrupt_to_break = aux["interrupt_to_break"]
    
    tinterrupt = interrupt_to_break + interrupt_to_work
    
    print('trabajando -> descansando',work_to_break/twork)
    print('trabajando -> interrupcion',work_to_interrupt/twork) 
    print()
    
    print('descanso -> interrupcion', break_to_interrupt/tbreak)
    print('descansando -> trabajando', break_to_work/tbreak)
    print()
    
    print('interrumpido -> descansando', interrupt_to_break/tinterrupt)
    print('interrumpido -> trabajando', interrupt_to_work/tinterrupt)
    
    
    
    return prob
   
    

In [62]:
get_prob(1000,480)

Exception ignored in: <generator object Person.working at 0x000001C9A918B9C0>
Traceback (most recent call last):
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\stride_tricks.py", line 538, in broadcast_arrays
    args = [np.array(_m, copy=False, subok=subok) for _m in args]
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object Person.working at 0x000001C9A8F8CE00>
Traceback (most recent call last):
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\stride_tricks.py", line 542, in <genexpr>
    if all(array.shape == shape for array in args):

RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object Person.working at 0x000001C9A8F8E0C0>
Traceback (most recent call last):
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\stride_tricks.py", line 542, in <gen

Index(['work_to_work', 'work_to_break', 'work_to_interrupt', 'break_to_work',
       'break_to_interrupt', 'interrupt_to_work', 'interrupt_to_break'],
      dtype='object')
trabajando -> descansando 0.27528568991983626
trabajando -> interrupcion 0.7247143100801637

descanso -> interrupcion 0.375283384019906
descansando -> trabajando 0.624716615980094

interrumpido -> descansando 0.185792499315631
interrumpido -> trabajando 0.814207500684369


,work_to_work,work_to_break,work_to_interrupt,break_to_work,break_to_interrupt,interrupt_to_work,interrupt_to_break
0,7,7,29,7,9,29,9
0,11,16,28,16,9,28,9
0,6,11,36,11,6,36,6
0,11,12,33,12,5,33,5
0,10,13,29,13,10,29,10
...,...,...,...,...,...,...,...
0,10,5,30,5,11,30,11
0,12,10,33,10,11,33,11
0,15,11,31,11,2,31,2
0,6,12,38,12,3,38,3


In [74]:
import numpy as np

# Definición de las probabilidades de transición basadas en los datos proporcionados
P = np.array([
    [0, 0.27528568991983626, 0.7247143100801637],
    [0.624716615980094, 0, 0.375283384019906],
    [0.814207500684369, 0.185792499315631, 0]
])

#variables contadoras
interruptions = 0
completed_tasks = 1
breaks = 0

# Estado inicial del trabajador
state = 'W'

# Número de pasos de tiempo para la simulación
Num = 100

# Lista para almacenar los estados a lo largo de la simulación
simulation = []

for _ in range(Num):
    if state == 'W':
        r = np.random.rand()
        if r < P[0, 0]:
            state = 'W'
        elif r < P[0, 0] + P[0, 1]:
            state = 'D'
        else:
            state = 'I'
    elif state == 'D':
        r = np.random.rand()
        if r < P[1, 0]:
            state = 'W'
        else:
            state = 'I'
    else:  # Estado I
        r = np.random.rand()
        if r < P[2, 0]:
            state = 'W'
        else:
            state = 'D'

    if state == 'W':
        completed_tasks += 1
    elif state == 'D':
        breaks += 1
    else:
        interruptions += 1
         

        
    simulation.append(state)

# Imprimir los resultados de la simulación
print("Estados a lo largo de la simulación:", simulation)
print(completed_tasks, breaks, interruptions)
total = completed_tasks + breaks + interruptions
print(completed_tasks/total, breaks/total, interruptions/total)


Estados a lo largo de la simulación: ['I', 'W', 'D', 'W', 'D', 'W', 'I', 'W', 'I', 'W', 'I', 'D', 'I', 'D', 'W', 'I', 'W', 'I', 'W', 'D', 'W', 'I', 'W', 'D', 'I', 'D', 'W', 'I', 'W', 'D', 'I', 'W', 'I', 'W', 'D', 'W', 'I', 'W', 'I', 'D', 'I', 'D', 'W', 'I', 'D', 'W', 'I', 'W', 'D', 'I', 'W', 'I', 'W', 'D', 'I', 'W', 'I', 'D', 'I', 'D', 'I', 'D', 'I', 'W', 'I', 'W', 'I', 'W', 'I', 'W', 'I', 'D', 'W', 'D', 'W', 'I', 'D', 'I', 'W', 'I', 'D', 'I', 'W', 'D', 'W', 'D', 'W', 'I', 'W', 'I', 'W', 'I', 'W', 'I', 'W', 'I', 'W', 'D', 'W', 'I']
39 24 38
0.38613861386138615 0.2376237623762376 0.37623762376237624


In [39]:
import copy
random.seed(RANDOM_SEED)

def simulate(n, time, b_time=0, b_duration=0, interruptions=True, verbose=False, likelihood=False, MSE=[5,5,10,10,30,30,30,30]):
    df = pd.DataFrame({})
    
    i=0
    invalid = likelihood
    while invalid or i < n:
        env = simpy.Environment()
        person = Person(env,'S', b_time, b_duration,interr= interruptions)
        env.run(until=time)
        env.event().succeed()
        env.event().succeed()
        env.event().succeed()
        # Data Collecting
        
        row = pd.DataFrame(
            {  
                "break_duration": [b_duration], 
                "break_timestamp": [b_time],
                "interruptions": [copy.copy(person.interrupts)],
                "completed_tasks": [copy.copy(person.completed_tasks)],
                "breaks": [copy.copy(person.breaks)],
                "working_time": [copy.copy(person.task_duration_sum)],
                "break_time": [copy.copy(person.break_duration_sum)],
                "interruption_time": [copy.copy(person.interruption_duration_sum)],
            }
        )
        
        
        df = pd.concat([df,row])
        
        # To verify stopping criteria
        if likelihood:
            est_std = df.std()
            # print(len(est_std[est_std/np.sqrt(i+1) < MSE]))
            invalid = len(est_std[est_std/np.sqrt(i+1) < MSE]) < 8
        

        # If the simulation verbosity is true the print the outcomes
        if verbose:
            print("Descanso: ",person.breaks)
            print("Tareas Completadas: ",person.completed_tasks)
            print("Duración Media de tareas: ",person.task_duration_sum/person.completed_tasks)
            print("Interrupciónes: ",person.interrupts)

        i+=1
    
    return df



## Extracting Data


In [75]:

df_rand= simulate(30, 480, likelihood=True)

aux = df_rand[['interruption_time','working_time','break_time','interruptions','breaks','completed_tasks']].apply(sum , axis=0)
total = (aux['interruption_time']/aux['interruptions']) + aux['working_time']/aux['completed_tasks'] + aux['break_time']/aux['breaks']
print(aux['working_time']/aux['completed_tasks']/total)
print(aux['break_time']/aux['breaks']/total)
print(aux['interruption_time']/aux['interruptions']/total)
df_rand.head()


Exception ignored in: <generator object Person.working at 0x000001C9A896E200>
Traceback (most recent call last):
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\generic.py", line 6320, in __setattr__
    object.__setattr__(self, name, value)
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object Person.working at 0x000001C9A8D24F40>
Traceback (most recent call last):
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\generic.py", line 6320, in __setattr__
    object.__setattr__(self, name, value)
RuntimeError: generator ignored GeneratorExit


Exception ignored in: <generator object Person.working at 0x000001C9A90CD940>
Traceback (most recent call last):
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\generic.py", line 6320, in __setattr__
    object.__setattr__(self, name, value)
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object Person.working at 0x000001C9A8D25F80>
Traceback (most recent call last):
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\stride_tricks.py", line 542, in <genexpr>
    if all(array.shape == shape for array in args):

RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object Person.working at 0x000001C9A8D27EC0>
Traceback (most recent call last):
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\stride_tricks.py", line 542, in <genexpr>
    if all(array.shape == shape for array in args):

RuntimeError: generator

0.4481387671270525
0.2850652057512765
0.26679602712167105


,break_duration,break_timestamp,interruptions,completed_tasks,breaks,working_time,break_time,interruption_time
0,0,0,35,6,9,12.772163,53.629325,133.636971
0,0,0,41,16,11,155.162651,28.793252,120.187672
0,0,0,40,9,16,33.662940,69.491835,94.120830
0,0,0,40,16,6,92.483778,25.308718,115.668954
0,0,0,41,8,9,32.032936,28.432869,107.861319
